## Import libraries

In [22]:
!pip install optuna

     |████████████████████████████████| 308 kB 2.7 MB/s            
     |████████████████████████████████| 1.6 MB 11.0 MB/s            
     |████████████████████████████████| 209 kB 8.4 MB/s            
     |████████████████████████████████| 80 kB 6.5 MB/s             
     |████████████████████████████████| 156 kB 9.7 MB/s            
     |████████████████████████████████| 149 kB 13.3 MB/s            
     |████████████████████████████████| 49 kB 7.7 MB/s             
     |████████████████████████████████| 112 kB 9.2 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=ad4cf075705055ccb63e1c60fe30b8786ca947565aa31cb34354ff959a56d60a
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [2]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import optuna
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.9.0a0+c3d40fd
Torchvision Version:  0.10.0a0


In [4]:
data_dir = "imagewoof2"
model_name = "resnet"
num_classes = 10
batch_size = 16
num_epochs = 300
use_pretrained = False
feature_extract = False
input_size = 224

## Train function for optuna

In [19]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return best_acc

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

In [7]:
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=10) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
criterion = nn.CrossEntropyLoss() 

## Model initialization

In [17]:
def define_model(trial):
    model_ft = models.resnet50(pretrained=use_pretrained)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)

    return model_ft

In [20]:
def objective(trial):
    model = define_model(trial).to(device)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    accuracy = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=200)
    
    return accuracy

## Tune hyperparameters

In [26]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-01-27 17:00:16,455] A new study created in memory with name: no-name-0ed9ac22-76f9-4a6a-8c73-8e3b118f00fe


Epoch 0/199
----------
train Loss: 5.8761 Acc: 0.1278
val Loss: 712.9957 Acc: 0.1064

Epoch 1/199
----------
train Loss: 2.3154 Acc: 0.1534
val Loss: 19.2550 Acc: 0.1250

Epoch 2/199
----------
train Loss: 2.3438 Acc: 0.1567
val Loss: 2.1834 Acc: 0.1924

Epoch 3/199
----------
train Loss: 2.2679 Acc: 0.1516
val Loss: 2.2280 Acc: 0.1792

Epoch 4/199
----------
train Loss: 2.2228 Acc: 0.1611
val Loss: 2.1807 Acc: 0.1754

Epoch 5/199
----------
train Loss: 2.2022 Acc: 0.1683
val Loss: 2.8004 Acc: 0.1074

Epoch 6/199
----------
train Loss: 2.2846 Acc: 0.1756
val Loss: 2.1515 Acc: 0.1812

Epoch 7/199
----------
train Loss: 2.1984 Acc: 0.1675
val Loss: 2.8160 Acc: 0.1234

Epoch 8/199
----------
train Loss: 2.1887 Acc: 0.1730
val Loss: 2.1837 Acc: 0.1774

Epoch 9/199
----------
train Loss: 2.1734 Acc: 0.1784
val Loss: 2.2389 Acc: 0.1726

Epoch 10/199
----------
train Loss: 2.1577 Acc: 0.1754
val Loss: 2.1372 Acc: 0.1960

Epoch 11/199
----------
train Loss: 2.1503 Acc: 0.1883
val Loss: 2.3200 

[I 2022-01-27 21:06:31,139] Trial 0 finished with value: 0.7548994655128531 and parameters: {'optimizer': 'RMSprop', 'lr': 0.04366056970214902}. Best is trial 0 with value: 0.7548994655128531.


val Loss: 2.8996 Acc: 0.3668

Training complete in 246m 13s
Best val Acc: 0.754899
Epoch 0/199
----------
train Loss: 2.3512 Acc: 0.1569
val Loss: 2.6536 Acc: 0.1542

Epoch 1/199
----------
train Loss: 2.1979 Acc: 0.1685
val Loss: 3.4213 Acc: 0.1586

Epoch 2/199
----------
train Loss: 2.1753 Acc: 0.1781
val Loss: 2.1355 Acc: 0.1889

Epoch 3/199
----------
train Loss: 2.1402 Acc: 0.1965
val Loss: 2.1914 Acc: 0.1932

Epoch 4/199
----------
train Loss: 2.1158 Acc: 0.2064
val Loss: 2.0893 Acc: 0.2194

Epoch 5/199
----------
train Loss: 2.0883 Acc: 0.2213
val Loss: 2.0332 Acc: 0.2301

Epoch 6/199
----------
train Loss: 2.0760 Acc: 0.2240
val Loss: 2.0802 Acc: 0.2339

Epoch 7/199
----------
train Loss: 2.0509 Acc: 0.2335
val Loss: 2.0218 Acc: 0.2522

Epoch 8/199
----------
train Loss: 2.0260 Acc: 0.2486
val Loss: 1.9450 Acc: 0.2907

Epoch 9/199
----------
train Loss: 1.9916 Acc: 0.2620
val Loss: 1.9310 Acc: 0.2975

Epoch 10/199
----------
train Loss: 1.9676 Acc: 0.2728
val Loss: 1.9120 Acc: 

[I 2022-01-28 01:39:36,330] Trial 1 finished with value: 0.8302367014507508 and parameters: {'optimizer': 'Adam', 'lr': 0.0033728675244193286}. Best is trial 1 with value: 0.8302367014507508.


...

[I 2022-01-28 09:12:23,316] Trial 3 finished with value: 0.30949350979893103 and parameters: {'optimizer': 'SGD', 'lr': 0.00011942103885257488}. Best is trial 1 with value: 0.8302367014507508.


val Loss: 1.9167 Acc: 0.3034

Training complete in 213m 58s
Best val Acc: 0.309494
Epoch 0/199
----------
train Loss: 2.2132 Acc: 0.1771
val Loss: 2.1811 Acc: 0.1670

Epoch 1/199
----------
train Loss: 2.1336 Acc: 0.2144
val Loss: 2.1599 Acc: 0.1855

Epoch 2/199
----------
train Loss: 2.1020 Acc: 0.2232
val Loss: 2.1074 Acc: 0.2288

Epoch 3/199
----------
train Loss: 2.0790 Acc: 0.2417
val Loss: 2.0637 Acc: 0.2550

Epoch 4/199
----------
train Loss: 2.0440 Acc: 0.2547
val Loss: 2.1191 Acc: 0.2331

Epoch 5/199
----------
train Loss: 2.0177 Acc: 0.2702
val Loss: 1.9657 Acc: 0.2973

Epoch 6/199
----------
train Loss: 1.9933 Acc: 0.2792
val Loss: 1.9419 Acc: 0.3013

Epoch 7/199
----------
train Loss: 1.9622 Acc: 0.2998
val Loss: 1.8972 Acc: 0.3204

Epoch 8/199
----------
train Loss: 1.9438 Acc: 0.3090
val Loss: 1.9640 Acc: 0.3085

Epoch 9/199
----------
train Loss: 1.9335 Acc: 0.3084
val Loss: 1.8909 Acc: 0.3243

Epoch 10/199
----------
train Loss: 1.9111 Acc: 0.3221
val Loss: 1.8257 Acc: 

[I 2022-01-28 12:36:53,036] Trial 4 finished with value: 0.7803512344107916 and parameters: {'optimizer': 'RMSprop', 'lr': 2.550790889736528e-05}. Best is trial 1 with value: 0.8302367014507508.


val Loss: 0.8932 Acc: 0.7707

Training complete in 204m 29s
Best val Acc: 0.780351
Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.8302367014507508
  Params: 
    optimizer: Adam
    lr: 0.0033728675244193286


/opt/conda/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


## Train ResNet50 with parameters: optimizer-Adam, lr-0.003

### Default train function 

In [9]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [35]:
final_model = models.resnet50(pretrained=use_pretrained)
num_ftrs = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs, num_classes)
final_model = final_model.to(device)

optimizer_ft = optim.Adam(final_model.parameters(), lr=0.003)
final_model, hist = train_model(final_model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/299
----------
train Loss: 2.3652 Acc: 0.1514
val Loss: 2.2741 Acc: 0.1481
Epoch 1/299
----------
train Loss: 2.1923 Acc: 0.1784
val Loss: 2.1881 Acc: 0.2034
Epoch 2/299
----------
train Loss: 2.1511 Acc: 0.1965
val Loss: 2.1019 Acc: 0.2204
Epoch 3/299
----------
train Loss: 2.1859 Acc: 0.1797
val Loss: 2.1214 Acc: 0.1950
Epoch 4/299
----------
train Loss: 2.1336 Acc: 0.2081
val Loss: 2.0650 Acc: 0.2347
Epoch 5/299
----------
train Loss: 2.0821 Acc: 0.2245
val Loss: 2.0566 Acc: 0.2311
Epoch 6/299
----------
train Loss: 2.0578 Acc: 0.2327
val Loss: 1.9403 Acc: 0.2830
Epoch 7/299
----------
train Loss: 2.0173 Acc: 0.2578
val Loss: 2.0294 Acc: 0.2614
Epoch 8/299
----------
train Loss: 1.9962 Acc: 0.2654
val Loss: 1.9318 Acc: 0.2952
Epoch 9/299
----------
train Loss: 1.9494 Acc: 0.2849
val Loss: 2.0381 Acc: 0.2606
Epoch 10/299
----------
train Loss: 1.9010 Acc: 0.3014
val Loss: 1.8561 Acc: 0.3047
Epoch 11/299
----------
train Loss: 1.8827 Acc: 0.3217
val Loss: 1.8131 Acc: 0.3464
Ep

In [39]:
torch.save(final_model.state_dict(), "/app/50.pth")

## Check the stability of other models to learning

### ResNet34

In [40]:
final_model = models.resnet34(pretrained=use_pretrained)
num_ftrs = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs, num_classes)
final_model = final_model.to(device)

optimizer_ft = optim.Adam(final_model.parameters(), lr=0.003)
final_model, hist = train_model(final_model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/299
----------
train Loss: 2.3294 Acc: 0.1499
val Loss: 2.4557 Acc: 0.1514
Epoch 1/299
----------
train Loss: 2.2108 Acc: 0.1754
val Loss: 2.3319 Acc: 0.1896
Epoch 2/299
----------
train Loss: 2.1736 Acc: 0.1787
val Loss: 2.1215 Acc: 0.1957
Epoch 3/299
----------
train Loss: 2.1514 Acc: 0.1958
val Loss: 2.0643 Acc: 0.2288
Epoch 4/299
----------
train Loss: 2.1226 Acc: 0.2088
val Loss: 2.2037 Acc: 0.1932
Epoch 5/299
----------
train Loss: 2.0799 Acc: 0.2265
val Loss: 1.9425 Acc: 0.2721
Epoch 6/299
----------
train Loss: 2.0471 Acc: 0.2408
val Loss: 1.9368 Acc: 0.2812
Epoch 7/299
----------
train Loss: 1.9992 Acc: 0.2688
val Loss: 2.0167 Acc: 0.2634
Epoch 8/299
----------
train Loss: 1.9556 Acc: 0.2831
val Loss: 1.8845 Acc: 0.3118
Epoch 9/299
----------
train Loss: 1.8885 Acc: 0.3165
val Loss: 1.7762 Acc: 0.3563
Epoch 10/299
----------
train Loss: 1.8269 Acc: 0.3465
val Loss: 1.6711 Acc: 0.4001
Epoch 11/299
----------
train Loss: 1.7643 Acc: 0.3732
val Loss: 1.8088 Acc: 0.3777
Ep

### ResNet18

In [10]:
final_model = models.resnet18(pretrained=use_pretrained)
num_ftrs = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs, num_classes)
final_model = final_model.to(device)

optimizer_ft = optim.Adam(final_model.parameters(), lr=0.003)
final_model, hist = train_model(final_model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)
torch.save(final_model.state_dict(), "/app/18.pth")

Epoch 0/299
----------


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 2.3041 Acc: 0.1572
val Loss: 2.1571 Acc: 0.1891
Epoch 1/299
----------
train Loss: 2.1752 Acc: 0.1875
val Loss: 2.0998 Acc: 0.2202
Epoch 2/299
----------
train Loss: 2.1250 Acc: 0.2104
val Loss: 2.0400 Acc: 0.2400
Epoch 3/299
----------
train Loss: 2.0698 Acc: 0.2382
val Loss: 1.9517 Acc: 0.2818
Epoch 4/299
----------
train Loss: 2.0186 Acc: 0.2604
val Loss: 1.8734 Acc: 0.3148
Epoch 5/299
----------
train Loss: 1.9540 Acc: 0.2916
val Loss: 1.8537 Acc: 0.3248
Epoch 6/299
----------
train Loss: 1.8788 Acc: 0.3202
val Loss: 1.7638 Acc: 0.3762
Epoch 7/299
----------
train Loss: 1.8070 Acc: 0.3488
val Loss: 1.6929 Acc: 0.3920
Epoch 8/299
----------
train Loss: 1.7434 Acc: 0.3796
val Loss: 1.6303 Acc: 0.4261
Epoch 9/299
----------
train Loss: 1.6925 Acc: 0.4009
val Loss: 1.5190 Acc: 0.4548
Epoch 10/299
----------
train Loss: 1.6362 Acc: 0.4193
val Loss: 1.4915 Acc: 0.4894
Epoch 11/299
----------
train Loss: 1.5818 Acc: 0.4417
val Loss: 1.4597 Acc: 0.4882
Epoch 12/299
----------
t

## Create requirements.txt

In [11]:
!pip freeze > requirements.txt

### Prepare model for deploing

In [13]:
model = models.resnet50(pretrained=use_pretrained)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model.load_state_dict(torch.load("/app/50.pth"))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
import torch.quantization
quantized_model = torch.quantization.quantize_dynamic(model.cpu(), {torch.nn.Linear}, dtype=torch.qint8)

In [15]:
torchscript_model = torch.jit.script(quantized_model)

In [16]:
torch.save(torchscript_model.state_dict(), "/app/50_torchscript.pth")

In [ ]:
torchscript_model(image)